### **Importing Libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.utils.data import Dataset
import torchvision.models as models
from torchvision import transforms
import torchvision
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchmetrics.segmentation import MeanIoU
import torchvision.transforms as T
from PIL import Image
import math

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### **Dataset**

#### **Helper Functions**

In [2]:
transform_base = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

transform_color = T.Compose([
    T.Resize((256, 256)),
    T.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_affine = T.Compose([
    T.Resize((256, 256)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=0.2),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def angle_to_vector(theta_deg):
    theta_rad = math.radians(theta_deg)
    return torch.tensor([math.cos(theta_rad), math.sin(theta_rad)], dtype=torch.float32)

def vector_to_angle(vector):
    cos_theta, sin_theta = vector
    angle_rad = torch.atan2(sin_theta, cos_theta)
    angle_deg = angle_rad * (180 / math.pi)
    # Ensure angle is in [0, 360) range
    return (angle_deg + 360) % 360

#### **Dataset Class**

In [3]:
class AngleDataset(Dataset):
    def __init__(self, image_dir, labels_df, transform=None):
        self.image_dir = image_dir
        self.labels_df = labels_df
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        angle = float(row['angle'])
        angle_vector = angle_to_vector(angle)  # (cosθ, sinθ)
        return image, angle_vector

#### **Extend Dataset**

In [4]:
def create_extended_dataset(image_dir, labels_df):
    # Original dataset
    original_dataset = AngleDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_base
    )
    
    # Color jitter augmented dataset
    color_dataset = AngleDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_color
    )
    
    # # Affine transform augmented dataset
    affine_dataset = AngleDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_affine
    )
    
    extended_dataset = ConcatDataset([original_dataset, color_dataset, affine_dataset])
    
    return extended_dataset

#### **Training**

In [5]:
image_dir_train = "Dataset/Train/images_train"
labels_path_train = "Dataset/Train/labels_train.csv"

labels_df = pd.read_csv(labels_path_train)

train_dataset = create_extended_dataset(image_dir_train, labels_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

#### **Validation**

In [6]:
images_dir_val = "Dataset/Val/images_val"
labels_path_val = "Dataset/Val/labels_val.csv"
labels_df_val = pd.read_csv(labels_path_val)

val_dataset = AngleDataset(images_dir_val, labels_df_val, transform_val)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

### **Model**

#### **Model Implementation**

In [7]:
class ResNetAngleRegressor(nn.Module):
    def __init__(self, pretrained=True, dropout_rate=0.3):
        super().__init__()
        base_model = models.resnet50(weights='DEFAULT' if pretrained else None)
        num_features = base_model.fc.in_features
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        
        # New regression head with dropout and batch normalization
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_features, 1024),
            nn.BatchNorm1d(1024), 
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            # nn.Linear(512, 256),
            # nn.BatchNorm1d(256),
            # nn.ReLU(),
            # nn.Dropout(dropout_rate),
            
            nn.Linear(1024, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.regressor(x)
        # Normalize output to enforce unit vector
        return x / torch.norm(x, dim=1, keepdim=True)

#### **Loss Function**

In [8]:
class AngleLoss(nn.Module):
    def __init__(self, reduction='mean'):
        super().__init__()
        self.reduction = reduction
        
    def forward(self, y_pred, y_true):

        y_pred_normalized = y_pred / torch.norm(y_pred, dim=1, keepdim=True)
        y_true_normalized = y_true / torch.norm(y_true, dim=1, keepdim=True)
        
        cos_angle_diff = torch.sum(y_pred_normalized * y_true_normalized, dim=1)
        
        cos_angle_diff = torch.clamp(cos_angle_diff, -1.0 + 1e-7, 1.0 - 1e-7)
        
        angle_loss = 1.0 - cos_angle_diff
        
        if self.reduction == 'mean':
            return angle_loss.mean()
        elif self.reduction == 'sum':
            return angle_loss.sum()
        else:
            return angle_loss

### **Training**

In [9]:
def train_regression_model(model, train_loader, val_loader, optimizer, num_epochs, device, loss_fn):
    model.to(device)
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        for images, targets in pbar:
            images = images.to(device)
            targets = targets.to(device)  # shape: (batch_size, 2)

            preds = model(images)  # output shape: (batch_size, 2)
            loss = loss_fn(preds, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            pbar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / len(train_loader.dataset)

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
            for images, targets in pbar:
                images = images.to(device)
                targets = targets.to(device)

                preds = model(images)
                loss = loss_fn(preds, targets)

                val_loss += loss.item() * images.size(0)
                pbar.set_postfix(loss=loss.item())


        avg_val_loss = val_loss / len(val_loader.dataset)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')

        print(f"Epoch {epoch+1}: Train MSE = {avg_train_loss:.4f}, Val MSE = {avg_val_loss:.4f}")

In [10]:
def evaluate_model(model, test_loader, device):
    model.eval()
    predictions = []
    ground_truths = []
    angle_errors = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            for i in range(outputs.size(0)):
                true_angle = vector_to_angle(labels[i]).item()
                pred_angle = vector_to_angle(outputs[i]).item()

                # Normalize angles to [0, 360)
                true_angle = true_angle % 360
                pred_angle = pred_angle % 360
                
                angle_diff = abs(true_angle - pred_angle)
                angle_diff = min(angle_diff, 360 - angle_diff)
                
                predictions.append(pred_angle)
                ground_truths.append(true_angle)
                angle_errors.append(angle_diff)
    
    mean_angle_error = sum(angle_errors) / len(angle_errors)
    median_angle_error = sorted(angle_errors)[len(angle_errors) // 2]
    
    mse = 0
    for i in range(len(predictions)):
        pred_rad = math.radians(predictions[i])
        true_rad = math.radians(ground_truths[i])
        
        pred_sin, pred_cos = math.sin(pred_rad), math.cos(pred_rad)
        true_sin, true_cos = math.sin(true_rad), math.cos(true_rad)
        
        mse += (pred_sin - true_sin)**2 + (pred_cos - true_cos)**2
    
    mse /= len(predictions)
    
    results = {
        'mean_angle_error': mean_angle_error,
        'median_angle_error': median_angle_error,
        'mse_sin_cos': mse,
        'predictions': predictions,
        'ground_truths': ground_truths,
        'angle_errors': angle_errors
    }
    return results

In [11]:
model = ResNetAngleRegressor()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
train_regression_model(model, train_loader, val_loader, optimizer, num_epochs=50, device=device, loss_fn=loss_fn)


Epoch 1/50 [Val]: 100%|██████████| 24/24 [00:02<00:00,  8.18it/s, loss=0.00592]


Epoch 1: Train MSE = 0.8538, Val MSE = 0.7338


Epoch 2/50 [Val]: 100%|██████████| 24/24 [00:02<00:00,  8.17it/s, loss=0.0864]


Epoch 2: Train MSE = 0.7043, Val MSE = 0.6471


Epoch 3/50 [Val]: 100%|██████████| 24/24 [00:02<00:00,  8.31it/s, loss=0.0139]


...

Epoch 48: Train MSE = 0.0468, Val MSE = 0.2491


Epoch 49/50 [Val]: 100%|██████████| 24/24 [00:02<00:00,  8.16it/s, loss=0.0117]


Epoch 49: Train MSE = 0.0433, Val MSE = 0.2372


Epoch 50/50 [Val]: 100%|██████████| 24/24 [00:02<00:00,  8.15it/s, loss=0.0303]

Epoch 50: Train MSE = 0.0453, Val MSE = 0.2419


In [12]:
# Evaluate the model on the validation set
results = evaluate_model(model, val_loader, device)
print(f"Mean Angle Error: {results['mean_angle_error']:.4f} degrees")
print(f"Median Angle Error: {results['median_angle_error']:.4f} degrees")
print(f"MSE (sin/cos): {results['mse_sin_cos']:.4f}")

Mean Angle Error: 31.3410 degrees
Median Angle Error: 20.2693 degrees
MSE (sin/cos): 0.4838


In [12]:
# Load the best model for evaluation

model_ev = ResNetAngleRegressor()
model_ev.to(device)
model_ev.load_state_dict(torch.load('best_model.pth'))

# Evaluate the model on the validation set
results = evaluate_model(model_ev, val_loader, device)
print(f"Mean Angle Error: {results['mean_angle_error']:.4f} degrees")
print(f"Median Angle Error: {results['median_angle_error']:.4f} degrees")
print(f"MSE (sin/cos): {results['mse_sin_cos']:.4f}")

Mean Angle Error: 30.8790 degrees
Median Angle Error: 19.3219 degrees
MSE (sin/cos): 0.4744
